# Prediction of housing prices using Random Forest Regression

## Importing libraries

In [1]:
import copy

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt

## Reading datasets

In [2]:
TRAINING_SET_PATH = '../dataset/train.csv'
training_set = pd.read_csv(TRAINING_SET_PATH)

TESTING_SET_PATH = '../dataset/test.csv'
testing_set = pd.read_csv(TESTING_SET_PATH)

## Data Preprocessing

In [ ]:
# Combining both sets to process together
print(f'Training set length: {len(training_set)}; Testing set lenth: {len(testing_set)}')
total_set = pd.concat([training_set, testing_set]).reset_index()
print(f'Total set length: {len(total_set)}')
testing_min_id = 1461

### Handling missing values

In [ ]:
def display_missing_values(df):
    missing_values = df.isnull().sum().to_frame()
    missing_values.columns = ['Missing_Values']
    missing_values = missing_values.query('Missing_Values > 0')
    print(missing_values)

display_missing_values(total_set)

In [5]:
cleaned_set = copy.deepcopy(total_set)

#### Handling Bsmt

Upon further inspection in the dataset, rows with NA *BsmtQual* have NA values for the rest of *BsmtColumns*

It would be suitable to create a new column of binary value to indicate whether the row has a *Bsmt*

In [ ]:
# Inspecting rows with NA BsmtQual
mask = total_set['BsmtQual'].isna()

cat_cols = ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2')
num_cols = ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath')
cols = cat_cols+num_cols

total_set.loc[mask, cols]

In [ ]:
# Inspecting rows with non-NA BsmtQual
mask = total_set['BsmtQual'].notna()
total_set.loc[mask, cols]

In [ ]:
# Replaceing NA with an arbitrary value
mask = cleaned_set['BsmtQual'].isna()
cleaned_set.loc[mask, cat_cols] = str('NA')
cleaned_set.loc[mask, num_cols] = 0

display_missing_values(cleaned_set)

In [ ]:
# cleanup BsmtCond
mask = cleaned_set['BsmtCond'].isna()
cleaned_set.loc[mask, 'BsmtCond'] = str('NA')

# cleanup BsmtExposure
mask = cleaned_set['BsmtExposure'].isna()
cleaned_set.loc[mask, 'BsmtExposure'] = str('NA')

# cleanup BsmtFinType2
mask = cleaned_set['BsmtFinType2'].isna()
cleaned_set.loc[mask, 'BsmtFinType2'] = str('NA')

display_missing_values(cleaned_set)

#### Handling Garage

Upon further inspection in the dataset, rows with NA *GarageYrBlt* have NA values for the rest of *GarageColumns*

It would be suitable to create a new column of binary value to indicate whether the row has a garage

In [ ]:
# Inspecting rows with NA YrBlt
mask = cleaned_set['GarageYrBlt'].isna()
cols = ('GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'GarageCars', 'GarageArea')
cleaned_set.loc[mask, cols]

In [ ]:
#   Replaceing NA with an arbitrary value
mask = cleaned_set['GarageYrBlt'].isna()

#   Handling categorical features
cat_cols = ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond')
cleaned_set.loc[mask, cat_cols] = str('NA')

#   Handling numerical feature
num_cols = ('GarageYrBlt', 'GarageCars', 'GarageArea')
cleaned_set.loc[mask, num_cols] = 0

display_missing_values(cleaned_set)

#### Handling Alley

In [ ]:
mask = cleaned_set['Alley'].isna()
cleaned_set.loc[mask, 'Alley'] = str('NA')

display_missing_values(cleaned_set)

#### Handling MasVnrType

In [ ]:
mask = cleaned_set['MasVnrType'].isna()
cleaned_set.loc[mask, 'MasVnrType'] = str('None')
display_missing_values(cleaned_set)

#### Handling MasVnrArea

In [ ]:
mask = cleaned_set['MasVnrArea'].isna()
cleaned_set.loc[mask, 'MasVnrArea'] = 0
display_missing_values(cleaned_set)

#### Handling FireplaceQu

Same approach for *FireplaceQu*

In [ ]:
mask = cleaned_set['FireplaceQu'].isna()
cleaned_set.loc[mask, 'FireplaceQu'] = str('NA')
display_missing_values(cleaned_set)

#### Handling PoolQC

Same approach for *PoolQC*

In [ ]:
mask = cleaned_set['PoolQC'].isna()
cleaned_set.loc[mask, 'PoolQC'] = str('NA')
display_missing_values(cleaned_set)

#### Handling Fence

Same approach for *Fence*

In [ ]:
mask = cleaned_set['Fence'].isna()
cleaned_set.loc[mask, 'Fence'] = str('NA')
display_missing_values(cleaned_set)

#### Handling MiscFeature

Same approach for *MiscFeature*

In [ ]:
mask = cleaned_set['MiscFeature'].isna()
cleaned_set.loc[mask, 'MiscFeature'] = str('NA')
display_missing_values(cleaned_set)

#### Handling LotFrontage

Since not all housing properties have lot frontage, such as apartments, it would be better to preserve the implication in the original dataset, instead of making up an approximate value.

In [ ]:
mask = cleaned_set['LotFrontage'].isna()
cleaned_set.loc[mask, 'LotFrontage'] = 0

display_missing_values(cleaned_set)

#### Handling misc NAs

In [ ]:
mask = cleaned_set['MSZoning'].isna()
cleaned_set.loc[mask, 'MSZoning'] = str('NA')

mask = cleaned_set['Utilities'].isna()
cleaned_set.loc[mask, 'Utilities'] = str('NA')

mask = cleaned_set['Exterior1st'].isna()
cleaned_set.loc[mask, 'Exterior1st'] = str('Other')

mask = cleaned_set['Exterior2nd'].isna()
cleaned_set.loc[mask, 'Exterior2nd'] = str('Other')

mask = cleaned_set['Electrical'].isna()
cleaned_set.loc[mask, 'Electrical'] = str('NA')

mask = cleaned_set['KitchenQual'].isna()
cleaned_set.loc[mask, 'KitchenQual'] = str('NA')

mask = cleaned_set['Functional'].isna()
cleaned_set.loc[mask, 'Functional'] = str('NA')

mask = cleaned_set['SaleType'].isna()
cleaned_set.loc[mask, 'SaleType'] = str('Oth')

display_missing_values(cleaned_set)

### Standardising numerical features

In [ ]:
# Getting numerical columns
num_features = cleaned_set.select_dtypes(include=['number']).columns.drop('Id')
num_features

#### Plotting distributions of numerical features

In [ ]:
def plot_dfs(features):
    # Setting up subplots
    plot_per_row = 4
    rows = len(features) // plot_per_row
    fig, axes = plt.subplots(rows+1, plot_per_row, figsize=(20, 6*rows))
    axes = axes.flatten()

    # Plotting each subplot
    for i, feature in enumerate(features):
        ax = axes[i]
        ax.hist(cleaned_set[feature], bins=len(cleaned_set[feature].unique()))
        ax.set_title(f'Distribution of {feature}')
        ax.set_xlabel(feature)
        ax.set_ylabel('Frequency')

    # Turning off unused subplot
    for j in range(i+1, len(axes)):
        axes[j].axis('off')

plot_dfs(num_features)

#### Standardising suitable features

Obsered in the plot of each feature's distribution, and it seems the following features should be standardised:
*LotArea*, *BsmtFinSF1*, *TotalBsmtSF*, *1stFlrSF*, *GrLivArea*

In [ ]:
standardised_features = ['LotArea', 'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea']

cleaned_set.loc[:, standardised_features] = StandardScaler().fit_transform(cleaned_set[standardised_features])
testing_set.loc[:, standardised_features] = StandardScaler().fit_transform(testing_set[standardised_features])

plot_dfs(standardised_features)

#### Standardising target

In [ ]:
cleaned_set.loc[:, 'SalePrice'] = np.log1p(cleaned_set[['SalePrice']])
plt.plot(cleaned_set.loc[:, 'SalePrice'])

### Encoding categorical features

In [21]:
# Getting categorical columns
cat_cols = cleaned_set.select_dtypes(include=['str']).columns
cat_cols

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

#### Encoding categorical columns using OrdinalEncoder

It is noted that not all categorical columns are ordinal features.

Despite the added implication, it doesn't really matter that nominal values are treated as such.

In [22]:
ord_enc = OrdinalEncoder()
ord_enc.fit(cleaned_set.loc[:, cat_cols])
encoded_cat = ord_enc.transform(cleaned_set.loc[:, cat_cols])
encoded_cat

array([[4., 1., 1., ..., 1., 8., 4.],
       [4., 1., 1., ..., 1., 8., 4.],
       [4., 1., 1., ..., 1., 8., 4.],
       ...,
       [4., 1., 1., ..., 1., 8., 0.],
       [4., 1., 1., ..., 3., 8., 4.],
       [4., 1., 1., ..., 1., 8., 4.]], shape=(2919, 43))

In [23]:
# Decoding categorical columns
ord_enc.inverse_transform(encoded_cat)

array([['RL', 'Pave', 'NA', ..., 'NA', 'WD', 'Normal'],
       ['RL', 'Pave', 'NA', ..., 'NA', 'WD', 'Normal'],
       ['RL', 'Pave', 'NA', ..., 'NA', 'WD', 'Normal'],
       ...,
       ['RL', 'Pave', 'NA', ..., 'NA', 'WD', 'Abnorml'],
       ['RL', 'Pave', 'NA', ..., 'Shed', 'WD', 'Normal'],
       ['RL', 'Pave', 'NA', ..., 'NA', 'WD', 'Normal']],
      shape=(2919, 43), dtype=object)

#### Subtituting encoded categorical features into the dataset

In [24]:
cleaned_set.loc[:, cat_cols] = encoded_cat
cleaned_set.loc[:, cat_cols]

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,4.0,1.0,1.0,3.0,3.0,0.0,4.0,0.0,5.0,2.0,...,1.0,2.0,5.0,5.0,2.0,3.0,4.0,1.0,8.0,4.0
1,4.0,1.0,1.0,3.0,3.0,0.0,2.0,0.0,24.0,1.0,...,1.0,2.0,5.0,5.0,2.0,3.0,4.0,1.0,8.0,4.0
2,4.0,1.0,1.0,0.0,3.0,0.0,4.0,0.0,5.0,2.0,...,1.0,2.0,5.0,5.0,2.0,3.0,4.0,1.0,8.0,4.0
3,4.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,6.0,2.0,...,5.0,3.0,5.0,5.0,2.0,3.0,4.0,1.0,8.0,0.0
4,4.0,1.0,1.0,0.0,3.0,0.0,2.0,0.0,15.0,2.0,...,1.0,2.0,5.0,5.0,2.0,3.0,4.0,1.0,8.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,5.0,1.0,1.0,3.0,3.0,0.0,4.0,0.0,10.0,2.0,...,6.0,1.0,3.0,3.0,2.0,3.0,4.0,1.0,8.0,4.0
2915,5.0,1.0,1.0,3.0,3.0,0.0,4.0,0.0,10.0,2.0,...,4.0,3.0,5.0,5.0,2.0,3.0,4.0,1.0,8.0,0.0
2916,4.0,1.0,1.0,3.0,3.0,0.0,4.0,0.0,11.0,2.0,...,5.0,3.0,5.0,5.0,2.0,3.0,4.0,1.0,8.0,0.0
2917,4.0,1.0,1.0,3.0,3.0,0.0,4.0,0.0,11.0,2.0,...,6.0,1.0,3.0,3.0,2.0,3.0,2.0,3.0,8.0,4.0


## Seperating training and testing sets

In [25]:
mask = cleaned_set['Id'] < testing_min_id
train = copy.deepcopy(cleaned_set.loc[mask, :])

mask = cleaned_set['Id'] >= testing_min_id
test = copy.deepcopy(cleaned_set.loc[mask, :])

print(f'Training set length: {len(train)}; Testing set length: {len(test)}')

Training set length: 1460; Testing set length: 1459


## Building model

In [26]:
max_depth = 10
random_state = 0
regr = RandomForestRegressor(max_depth=max_depth, random_state=random_state)

regr = regr.fit(X=training_set.drop(['Id', 'SalePrice'], axis=1), y=training_set.loc[:, 'SalePrice'])
regr.score(X=training_set.drop(['Id', 'SalePrice'], axis=1), y=training_set.loc[:, 'SalePrice'])

ValueError: could not convert string to float: 'RL'

## Converting testing set

In [ ]:
y_pred = regr.predict(testing_set.drop('Id', axis=1))
y_true = pd.read_csv('../dataset/sample_submission.csv')

mean_squared_error(y_true, y_pred)